In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from db_utils import query_hive_ssh
import re
import copy
import time
import numpy as np
import os


In [2]:
def transfer_table(params, dry = False):
    hdfs_path = '/user/hive/warehouse/%(db)s.db/%(table)s' % params
    stat2_path = '/home/ellery/detox/data/retention/%(name)s' % params
    local_path = '/Users/ellerywulczyn/detox/data/retention/%(name)s/' % params
    

    if not dry:
        # transfer from HDFS to stat2
        cmd = "ssh stat1002.eqiad.wmnet 'rm -rf %s'" % stat2_path
        print(os.system(cmd))
        cmd = "ssh stat1002.eqiad.wmnet 'hadoop fs -copyToLocal %s %s '" % (hdfs_path, stat2_path)
        print(os.system(cmd))
        #transfer from stat2 to local
        cmd = 'rm -rf %s' % local_path
        print(os.system(cmd))
        cmd = 'rsync -avz  stat1002.eqiad.wmnet:%s/* %s' % (stat2_path, local_path)
        os.system(cmd)
        
        
    dfs = []

    for filename in os.listdir(local_path):
        try:
            dfs.append(pd.read_csv(os.path.join(local_path,filename), sep = '\t', header = None))
        except:
            print("Error reading: ", os.path.join(local_path,filename))

    df = pd.concat(dfs)
    df.columns = params['columns']

    df = df[df['user_text'] != "false"]

    df.to_csv(local_path[:-1] + '.tsv', sep ='\t', index = False)

In [8]:
cmd = "SELECT * from enwiki.daily_revision_counts limit 10;"
from db_utils import query_hive_ssh
query_hive_ssh(cmd, '/tmp/testfile')

,daily_revision_counts.user_id,daily_revision_counts.user_text,daily_revision_counts.day,daily_revision_counts.ns,daily_revision_counts.n_revisions,daily_revision_counts.n_deleted_revisions,daily_revision_counts.n_identity_reverted_revisions,daily_revision_counts.n_productive_revisions
0,1,Damian Yerrick,20041121,0,1,0,0,1
1,1,Damian Yerrick,20050919,1,1,0,0,1
2,1,Damian Yerrick,20060227,0,1,0,0,1
3,1,Damian Yerrick,20070829,0,3,0,0,3
4,1,Damian Yerrick,20110818,0,2,0,0,2
5,2,AxelBoldt,20030608,0,24,0,1,23
6,2,AxelBoldt,20060211,3,4,0,0,4
7,2,AxelBoldt,20060421,0,4,0,0,4
8,2,AxelBoldt,20071022,0,2,0,0,2
9,2,AxelBoldt,20080121,2,2,0,0,2


In [3]:
params = {
        'db': 'enwiki',
        'table':'daily_revision_counts',
        'name': 'daily_revision_counts',
        'columns': ['user_id','user_text','day','ns', 'n_revisions', 'n_deleted_revisions', 'n_identity_reverted_revisions', 'n_productive_revisions']
    }
transfer_table(params, dry = False)

0
0
0
Error reading:  /Users/ellerywulczyn/detox/data/retention/daily_revision_counts/000096_0
Error reading:  /Users/ellerywulczyn/detox/data/retention/daily_revision_counts/000251_0


In [11]:
params = {
        'db': 'newcomer',
        'table':'daily_revision_counts',
        'name': 'newcomer_daily_revision_counts',
        'columns': ['user_id','user_text','day','ns', 'n_revisions', 'n_deleted_revisions', 'n_identity_reverted_revisions', 'n_productive_revisions']
    }
transfer_table(params, dry = False)

0
0
0


In [4]:
params = {
        'db': 'enwiki',
        'table':'user_start',
        'name': 'user_start',
        'columns': ['user_id','user_text', 'registration_day', 'first_edit_day']
    }
transfer_table(params, dry = False)

0
0
0


In [12]:
params = {
        'db': 'newcomer',
        'table':'user_start',
        'name': 'newcomer_user_start',
        'columns': ['user_id','user_text', 'registration_day', 'first_edit_day']
    }
transfer_table(params, dry = False)

0
0
0


In [5]:
params = {
        'db': 'enwiki',
        'table':'user_warnings',
        'name': 'user_warnings',
        'columns': [
                    'rev_id',
                    'page_id',
                    'page_title',
                    'rev_timestamp',
                    'user_id',
                    'user_text',
                    'bot',
                    'admin',
                    'ns',
                    'year',
                    'type',
                    'level',
                    ]
    }
transfer_table(params, dry = False)

0
0
0


In [13]:
params = {
        'db': 'newcomer',
        'table':'user_warnings',
        'name': 'newcomer_user_warnings',
        'columns': [
                    'rev_id',
                    'page_id',
                    'page_title',
                    'rev_timestamp',
                    'user_id',
                    'user_text',
                    'bot',
                    'admin',
                    'ns',
                    'year',
                    'type',
                    'level',
                    ]
    }
transfer_table(params, dry = False)

0
0
0
